## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
from PIL import Image
import glob
import cv2
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'
model_dir = '/opt/ml/model'

## 1. Model 정의

In [3]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 2. Training Dataset 정의

In [4]:
class TrainDataset(Dataset):
    def __init__(self, data_root, transform):
        self.data_root = data_root
        self.transform = transform
        self.imag_list = self._load_img_list(data_root)

    def __getitem__(self, index):
        img_path = self.imag_list[index]
        img = Image.open(img_path)
        
        if self.transform:
            img = self.transform(img)

        # Ground Truth
        label = self._get_class_idx_from_img_name(img_path)

        return img, label

    def __len__(self):
        return len(self.imag_list)
    
    def _load_img_list(self, data_root):
        img_list = []
        image_dir = os.path.join(data_root, 'images')
        
        for dir in glob.glob(image_dir + '/*'):
            img_list.extend(glob.glob(dir+'/*'))

        return img_list

    def _load_img_ID(self, img_path):
        return img_path.split('/')[7].split('_')[0]

    def _get_class_idx_from_img_name(self, img_path):
        img_name = os.path.basename(img_path)
        img_id = self._load_img_ID(img_path)
        
        img_idx = train_data.loc[train_data['id'] == img_id].index
        v = train_data.iloc[img_idx]['age+gender'].values[0]
        if 'normal' in img_name:
            return 12 + v
        elif 'incorrect_mask' in img_name:
            return 6 + v
        else:
            return 0 + v


## 3. Training

In [5]:
# meta 데이터와 이미지 경로를 불러옵니다.
train_data = pd.read_csv(os.path.join(train_dir, 'train_1.csv'))

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

dataset = TrainDataset(train_dir, transform)

loader = DataLoader(
    dataset,
    batch_size = 32,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = MyModel(num_classes=18).to(device)
# model.train(loader)

In [6]:
# Loss function and Optimizer
from torch.optim import Adam

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

In [7]:
# Training
epochs = 5

for epoch in range(epochs):
    for iter, (img, label) in tqdm(enumerate(loader)):
        # optimizer에 저장된 미분값을 0으로 초기화
        optimizer.zero_grad()

        # GPU 연산을 위해 이미지와 정답 tensor를 GPU로 보내기 (필요한 경우, 변수의 type도 수정해주세요)
        img, label = img.to(device), label.to(device)

        # 모델에 이미지 forward
        pred_logit = model.forward(img)
        # print(pred_logit, label)

        # loss 값 계산
        loss = criterion(pred_logit, label)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Accuracy 계산
        pred_label = torch.argmax(pred_logit, dim=1)
        acc = torch.sum((pred_label == label).float()) / len(img)

        train_loss = loss.item()
        train_acc = acc

591it [07:52,  1.25it/s]
591it [07:49,  1.26it/s]
591it [07:48,  1.26it/s]
591it [07:56,  1.24it/s]
591it [07:56,  1.24it/s]
591it [07:53,  1.25it/s]
591it [07:55,  1.24it/s]
591it [07:56,  1.24it/s]
591it [07:55,  1.24it/s]
591it [07:56,  1.24it/s]


In [8]:
torch.save(model.state_dict(), model_dir + '/save')